In [1]:
import os

In [2]:
os.chdir('../')


In [48]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    all_columns: dict
    metric_file_name: Path
    target_column: str

In [58]:
from NBA_Project.constants import *
from src.NBA_Project.utils.common import read_yaml,create_directories,save_json


In [59]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN
        columns=self.schema.COLUMNS

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            all_columns=columns
           
        )

        return model_evaluation_config

In [60]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import joblib


In [61]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self,actual,pred):

        precision=precision_score(actual,pred)
        recall=recall_score(actual,pred)
        f1=f1_score(actual,pred)

        return precision,recall,f1
    
    def save_results(self):

        schema=self.config.all_columns

        column_list = list(schema.keys())


        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_x = test_x[column_list]
        print(test_x.columns)
        test_y = test_data[[self.config.target_column]]

        predicted_perf = model.predict(test_x)
        (precision,recall,f1)=self.eval_metrics(test_y, predicted_perf)
        scores = {"Precision": precision, "Recall": recall, "f1_score": f1}


        save_json(path=Path(self.config.metric_file_name), data=scores)



In [62]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2023-10-21 19:40:30,011: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-21 19:40:30,013: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-21 19:40:30,015: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-21 19:40:30,017: INFO: common: created directory at artifacts]
[2023-10-21 19:40:30,017: INFO: common: created directory at artifacts/model_evaluation]
Index(['GP', 'MIN', 'FTM', '3PA', 'OREB', 'BLK', '3P Made', '3P%', 'FG%'], dtype='object')


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- AST
- DREB
- FGA
- FGM
- FT%
- ...
